In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam



def vgg_block(nbr_de_filter, nbr_de_conv_layer, shape, model, *input_shape):

  if len(input_shape) == 0:
      for _ in range(nbr_de_conv_layer):
        model.add(Conv2D(filters=nbr_de_filter, kernel_size=(shape, shape), padding='same',activation='relu'))
        model.add(BatchNormalization())
      model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
      model.add(BatchNormalization())
  else:
      model.add(Conv2D(input_shape=input_shape[0],filters=nbr_de_filter,kernel_size=(shape, shape),padding="same", activation="relu"))
      model.add(BatchNormalization())
      for _ in range(nbr_de_conv_layer-1):
        model.add(Conv2D(filters=nbr_de_filter, kernel_size=(shape, shape), padding='same',activation='relu'))
        model.add(BatchNormalization())
      model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
      model.add(BatchNormalization())
  
  return model

In [ ]:
def vgg_network(conv_architect, input_shape):
  network = Sequential()

  # Conv blocks
  compt = 0
  for (nbr_de_filter, nbr_de_conv_layer, shape) in conv_architect:
    if compt == 0:
      network = vgg_block(nbr_de_filter,nbr_de_conv_layer, shape, network,input_shape)
      compt=compt+1
    else:
      network = vgg_block(nbr_de_filter,nbr_de_conv_layer, shape, network)

  # The three fully connected layer
  network.add(Flatten())
  network.add(Dense(units=4096,activation="relu"))
  network.add(Dropout(0.5))
  
  # network.add(Flatten())
  network.add(Dense(units=4096,activation="relu"))
  network.add(Dropout(0.5))

  network.add(Dense(units=1000,activation="softmax"))
  opt = Adam(lr=0.001)
  network.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

  return network


In [ ]:
# definie nbr of filter, nbr de layer and shape of kernel in each block
conv_architect = [[64,2,3],[128, 2, 3], [256, 4, 3], [512, 4, 2], [512, 4, 3]]

In [ ]:
model = vgg_network(conv_architect,(224,224,3))
print(model.summary())